In [1]:
import time
import numpy as np

np.random.seed(0)

from lib.linear_algebra import Matrix
from lib.nn import Linear, ReLU, NN
from lib.processing import ColumnNormalizer
from lib.metrics.losses import mean_squared_error
from lib.gd_data_loaders import BatchDataLoader, StochasticDataLoader, MiniBatchDataLoader
from lib.optimizers import SgdOptimizer, SgdWithMomentumOptimizer, AdaGradOptimizer, RmsPropOptimizer, AdamOptimizer

In [2]:
# Boston House Price Dataset
# The Boston House Price Dataset involves the prediction of a house price in thousands of dollars given details of the house and its neighborhood.
# It is a regression problem. There are 506 observations with 13 input variables and 1 output variable. The variable names are as follows:

data = []
with open("data/boston_house_prices.data", "rt") as f:
    for line in f.readlines():
        data.append([float(v) for v in line.split()])
data = np.array(data)

In [3]:
np.random.shuffle(data)
split = int(len(data) * 0.8)
train_set = data[:split]
test_set = data[split:]

X_train, y_train = train_set[:, :-1], train_set[:, -1:]
X_test, y_test = test_set[:, :-1], test_set[:, -1:]
X_train = Matrix(X_train)
y_train = Matrix(y_train)
X_test = Matrix(X_test)
y_test = Matrix(y_test)
X_train.dims(), y_train.dims(), X_test.dims(), y_test.dims()

((404, 13), (404, 1), (102, 13), (102, 1))

In [4]:
normalizer = ColumnNormalizer()
normalizer.fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)
X_train.dims(), X_test.dims()

((404, 13), (102, 13))

In [5]:
def init_nn():
    return NN([
        Linear(13, 4),
        ReLU(),
        Linear(4, 1),
    ])

In [6]:
time_point = time.time()

data_loaders = [
    # BatchDataLoader(X_train, y_train),
    # StochasticDataLoader(X_train, y_train),
    MiniBatchDataLoader(X_train, y_train, 32)
]
optimizer_creators = [
    # lambda nn: SgdOptimizer(nn, 0.001),
    # lambda nn: SgdWithMomentumOptimizer(nn, 0.001, 0.9),
    lambda nn: AdaGradOptimizer(nn, 0.05),
    lambda nn: RmsPropOptimizer(nn, 0.01, 0.95),
    lambda nn: AdamOptimizer(nn, 0.01, 0.95, 0.95),
]

for data_loader in data_loaders:
    for optimizer_creator in optimizer_creators:
        nn = init_nn()
        optimizer = optimizer_creator(nn)
        print(f"gradient descent: {data_loader.__class__} | optimizer: {optimizer.__class__}")
        for i in range(2001):
            X_b, y_b = data_loader.get_batch()
            out = nn(X_b)
            loss = mean_squared_error(y_b, out)
            if i % 100 == 0:
                elapsed_time = int(time.time() - time_point)
                time_point = time.time()
                print(f"{i} | {loss.data:.2f} | {elapsed_time}s") 

            optimizer.step(loss)

        train_out = nn(X_train) 
        train_loss = mean_squared_error(y_train, train_out)
        test_out = nn(X_test) 
        test_loss = mean_squared_error(y_test, test_out)
        print(f"train loss: {train_loss.data:.2f}   test loss: {test_loss.data:.2f}") 

gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.optimizers.AdaGradOptimizer'>
0 | 675.96 | 0s
100 | 179.02 | 12s
200 | 59.88 | 11s
300 | 34.59 | 12s
400 | 30.84 | 12s
500 | 26.57 | 13s
600 | 15.89 | 13s
700 | 47.15 | 13s
800 | 39.12 | 12s
900 | 33.34 | 11s
1000 | 25.93 | 14s
1100 | 18.08 | 11s
1200 | 18.90 | 10s
1300 | 24.24 | 10s
1400 | 13.47 | 10s
1500 | 14.16 | 10s
1600 | 13.63 | 10s
1700 | 12.31 | 10s
1800 | 14.04 | 10s
1900 | 32.22 | 10s
2000 | 19.95 | 10s
train loss: 20.69   test loss: 20.87
gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.optimizers.RmsPropOptimizer'>
0 | 635.87 | 1s
100 | 119.93 | 13s
200 | 23.57 | 13s
300 | 13.51 | 13s
400 | 29.39 | 14s
500 | 8.80 | 14s
600 | 16.64 | 13s
700 | 11.79 | 14s
800 | 8.33 | 13s
900 | 20.61 | 13s
1000 | 24.01 | 13s
1100 | 12.77 | 14s
1200 | 18.11 | 13s
1300 | 12.45 | 13s
1400 | 9.86 | 14s
1500 | 13.91 | 13s
1600 | 10.44 | 13s
1700 | 8.45 | 14s
18

In [7]:
[(round(float(v1),1), float(v2)) for v1, v2 in zip([v[0].data for v in out.values], [v[0].data for v in y_test])][:10]

[(15.6, 20.7),
 (24.9, 39.8),
 (12.3, 17.8),
 (41.5, 19.6),
 (19.3, 14.9),
 (26.2, 22.0),
 (13.2, 48.8),
 (34.4, 25.0),
 (35.9, 48.5),
 (7.4, 23.9)]